In [ ]:
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.python.keras.models import Sequential,model_from_json
from tensorflow.python.keras.preprocessing import image
from tensorflow.keras.callbacks            import ReduceLROnPlateau, ModelCheckpoint
from tensorflow.python.keras.layers import Activation, Dropout, Flatten, Dense
from keras.applications import VGG16
from tensorflow.python.keras.optimizers import Adam
import numpy as np
import os 
from keras.preprocessing.image import img_to_array
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import os
import zipfile
import cv2
import shutil
import random
import pandas as pd
from matplotlib.image import imread
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

## UPLOADING FILES

---



In [ ]:
file = files.upload()
p = os.path.abspath('kaggle.json')
print(p)

In [ ]:
!pip install kaggle

In [ ]:
!mkdir ~/.kaggle
p = os.path.abspath('.kaggle')
print(p)
!ls

In [ ]:
!mv /content/kaggle.json ~/.kaggle
!kaggle competitions download -c dogs-vs-cats

In [ ]:
!unzip '/content/test1.zip'
!unzip '/content/train.zip'

## Data Preparation

---



In [ ]:
# Каталог с набором данных
data_dir = '/content/train'
# Каталог с данными для обучения
train_dir = 'train01'
test_sub = 'final-dataset'
# Каталог с данными для проверки
val_dir = 'val'
# Каталог с данными для тестирования
test_dir = 'test01'
# Часть набора данных для тестирования
test_data_portion = 0.15
# Часть набора данных для проверки
val_data_portion = 0.15
# Количество элементов данных в одном классе
nb_images = 12500

In [ ]:
def create_directory(dir_name):
    if os.path.exists(dir_name):
        shutil.rmtree(dir_name)
    os.makedirs(dir_name)
    os.makedirs(os.path.join(dir_name, "cats"))
    os.makedirs(os.path.join(dir_name, "dogs"))

In [ ]:
create_directory(train_dir)
create_directory(val_dir)
create_directory(test_sub)
create_directory(test_dir)

In [ ]:
def copy_images(start_index, end_index, source_dir, dest_dir):
    for i in range(start_index, end_index):
        shutil.copy2(os.path.join(source_dir, "cat." + str(i) + ".jpg"), 
                    os.path.join(dest_dir, "cats"))
        shutil.copy2(os.path.join(source_dir, "dog." + str(i) + ".jpg"), 
                   os.path.join(dest_dir, "dogs"))

In [ ]:
final_data = int(nb_images * 0.5)
start_val_data_idx = int(nb_images * (1 - val_data_portion - test_data_portion))
start_test_data_idx = int(nb_images * (1 - test_data_portion))
print(start_val_data_idx)
print(final_data)
print(start_test_data_idx)

In [ ]:
test = '/content/test1'
copy_images(0, final_data, data_dir, test_sub)
copy_images(0, start_val_data_idx, data_dir, train_dir)
copy_images(start_val_data_idx, start_test_data_idx, data_dir, val_dir)
copy_images(start_test_data_idx, nb_images, data_dir, test_dir)

In [ ]:
img_width, img_height = 150, 150
input_shape = (img_width, img_height, 3)
batch_size = 10
nb_train_samples = 17500
nb_validation_samples = 3750
nb_test_samples = 3750

In [ ]:
# define location of dataset
folder = '/content/train/'
# plot first few images
for i in range(9):
	# define subplot
	plt.subplot(330 + 1 + i)
	# define filename
	filename = folder + 'dog.' + str(i) + '.jpg'
	# load image pixels
	image = imread(filename)
	# plot raw pixel data
	plt.imshow(image)
# show the figure
plt.show()

## Creating model

---



---



In [ ]:
vgg16_net = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

In [ ]:
datagen = ImageDataGenerator(rescale=1. / 255)

In [ ]:
train_dir = '/content/train01'
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(img_width,img_height),
    batch_size=batch_size,
    class_mode=None,
    shuffle=False)

In [ ]:
val_generator = datagen.flow_from_directory(
    val_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode=None,
    shuffle=False)

In [ ]:
test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode=None,
    shuffle=False)

In [ ]:
features_train = vgg16_net.predict_generator(
        train_generator, 
        nb_train_samples // batch_size)
np.save(open('/content/features_train.npy', 'wb'), features_train)

In [ ]:
features_val = vgg16_net.predict_generator(
        val_generator, nb_validation_samples // batch_size)
np.save(open('/content/features_val.npy', 'wb'), features_val)

In [ ]:
features_test = vgg16_net.predict_generator(
        test_generator, nb_test_samples // batch_size)
np.save(open('/content/features_test.npy', 'wb'), features_test)

In [ ]:
labels_train =  np.array(
        [0] * (nb_train_samples // 2) + [1] * (nb_train_samples // 2))
labels_val =  np.array(
        [0] * (nb_validation_samples // 2) + [1] * (nb_validation_samples // 2))
labels_test =  np.array(
        [0] * (nb_test_samples // 2) + [1] * (nb_test_samples // 2))

In [ ]:
features_train = np.load(open('features_train.npy', 'rb'))
features_val = np.load(open('features_val.npy', 'rb'))
features_test = np.load(open('features_test.npy', 'rb'))

In [ ]:
def model_init_():
  model = Sequential()
  model.add(Flatten(input_shape=features_train.shape[1:]))
  model.add(Dense(512, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(1, activation='sigmoid'))
  return model
model = model_init_()

In [ ]:
model.compile(optimizer='Adam',
              loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
сheckpoint = ModelCheckpoint('content/mnist-cnn.h5', 
                              monitor='val_acc', 
                              save_best_only=True,
                              verbose=1)
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

In [ ]:
history = model.fit(features_train, labels_train,
              epochs=30,
              batch_size=64,
              validation_data=(features_val, labels_val), 
              verbose=2,
              callbacks=[сheckpoint, learning_rate_reduction])

In [ ]:
model.save('/content/model01.h5')

In [ ]:
def plot_model_history(model_history, acc='accuracy', val_acc='val_accuracy'):
    fig, axs = plt.subplots(1,2,figsize=(15,5))
    axs[0].plot(range(1,len(model_history.history[acc])+1),model_history.history[acc])
    axs[0].plot(range(1,len(model_history.history[val_acc])+1),model_history.history[val_acc])
    axs[0].set_title('Model Accuracy')
    axs[0].set_ylabel('Accuracy')
    axs[0].set_xlabel('Epoch')
    axs[0].set_xticks(np.arange(1,len(model_history.history[acc])+1),len(model_history.history[acc])/10)
    axs[0].legend(['train', 'val'], loc='best')
    axs[1].plot(range(1,len(model_history.history['loss'])+1),model_history.history['loss'])
    axs[1].plot(range(1,len(model_history.history['val_loss'])+1),model_history.history['val_loss'])
    axs[1].set_title('Model Loss')
    axs[1].set_ylabel('Loss')
    axs[1].set_xlabel('Epoch')
    axs[1].set_xticks(np.arange(1,len(model_history.history['loss'])+1),len(model_history.history['loss'])/10)
    axs[1].legend(['train', 'val'], loc='best')
    plt.show()
    
plot_model_history(history)

In [ ]:
scores = model.evaluate(features_test, labels_test, verbose=1)
print("Аккуратность на тестовых данных: %.2f%%" % (scores[1]*100))